In [1]:
# !pip3 install pinecone


In [2]:
# # Import the Pinecone library
# from pinecone import Pinecone

# # Initialize a Pinecone client with your API key
# pc = Pinecone(api_key="pcsk_4qt3ZT_U7hLvXByvnhPbktY96YWtg4vh4TQqURmWBWHuLJZquBFQTBNXaaayfbPo8G7ZmT")

# # Create a dense index with integrated embedding
# index_name = "quickstart-py"
# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
#         cloud="aws",
#         region="us-east-1",
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"}
#         }
#     )


In [3]:
# records = [
#     {
#         "_id": "rec1",
#         "chunk_text": "What is SAP used for? SAP is software that helps companies manage business activities like finance, sales, supply chain, and HR. It keeps all important data in one system so different teams can work together smoothly.",
#         "category": "SAP Overview"
#     },
#     {
#         "_id": "rec2",
#         "chunk_text": "What is SAP HANA? SAP HANA is a fast database that stores company information in memory. This makes reports and analytics much faster compared to traditional databases.",
#         "category": "SAP Overview"
#     },
#     {
#         "_id": "rec3",
#         "chunk_text": "What is the difference between SAP ECC and SAP S/4HANA? SAP ECC is the older version of SAP's ERP system. SAP S/4HANA is the newer version, designed to be faster and simpler, and it runs only on the HANA database.",
#         "category": "SAP Overview"
#     },
#     {
#         "_id": "rec4",
#         "chunk_text": "What does 'in-memory' mean in SAP HANA? It means data is stored in the computer's memory (RAM) instead of on a hard disk. This allows SAP to process data in real time, like generating instant reports.",
#         "category": "SAP Overview"
#     },
#     {
#         "_id": "rec5",
#         "chunk_text": "What is SAP Fiori? SAP Fiori is a modern, user-friendly way to use SAP applications. It looks like apps on your phone, making it easier to navigate compared to the older SAP screens.",
#         "category": "SAP Interface"
#     },
#     {
#         "_id": "rec6",
#         "chunk_text": "How does SAP keep data secure? SAP uses login credentials, role-based permissions, and data encryption. This ensures employees only see the information they need for their job.",
#         "category": "SAP Security"
#     },
#     {
#         "_id": "rec7",
#         "chunk_text": "How do I access the company email system? New employees can access the company email via Outlook using your corporate credentials. Set up instructions are in the IT onboarding guide.",
#         "category": "IT & Tools"
#     },
#     {
#         "_id": "rec8",
#         "chunk_text": "Who should I contact for HR-related questions? For questions about benefits, leave, or payroll, contact the HR team at hr@company.com or check the HR portal.",
#         "category": "HR"
#     },
#     {
#         "_id": "rec9",
#         "chunk_text": "How do I log into the internal SAP system? Use your employee ID and temporary password provided during onboarding. After first login, you’ll be prompted to set a new password.",
#         "category": "SAP Access"
#     },
#     {
#         "_id": "rec10",
#         "chunk_text": "Where can I find company policies and guidelines? All company policies are available in the internal employee portal under 'Documents & Policies'.",
#         "category": "Policies"
#     },
#     {
#         "_id": "rec11",
#         "chunk_text": "Who do I contact if I have technical issues? Contact the IT helpdesk at it-support@company.com or submit a ticket via the support portal for assistance.",
#         "category": "IT & Support"
#     }
# ]


In [4]:
# # Target the index
# dense_index = pc.Index(index_name)

# # Upsert the records into a namespace
# dense_index.upsert_records("example-namespace", records)


In [5]:
# # Wait for the upserted vectors to be indexed
# import time
# time.sleep(10)

# # View stats for the index
# stats = dense_index.describe_index_stats()
# print(stats)

In [6]:
# query = "How do I access SAP and company tools?"

# results = dense_index.search(
#     namespace="sap-onboarding",
#     query={
#         "top_k": 10,
#         "inputs": {
#             'text': query
#         }
#     }
# )

# for hit in results['result']['hits']:
#     print(
#         f"id: {hit['_id']:<5} | "
#         f"score: {round(hit['_score'], 2):<5} | "
#         f"category: {hit['fields']['category']:<15} | "
#         f"text: {hit['fields']['chunk_text']:<80}"
#     )


In [7]:
# # Search the dense index and rerank results
# reranked_results = dense_index.search(
#     namespace="example-namespace",
#     query={
#         "top_k": 10,
#         "inputs": {
#             'text': query
#         }
#     },
#     rerank={
#         "model": "bge-reranker-v2-m3",
#         "top_n": 10,
#         "rank_fields": ["chunk_text"]
#     }   
# )

# # Print the reranked results
# for hit in reranked_results['result']['hits']:
#     print(f"id: {hit['_id']}, score: {round(hit['_score'], 2)}, text: {hit['fields']['chunk_text']}, category: {hit['fields']['category']}")


In [8]:
# # Delete the index
# pc.delete_index(index_name)


openai 

In [9]:
# !pip3 install -q pinecone-client openai langchain langchain-text-splitters


In [10]:
# import time
# from typing import List, TypedDict

# from langchain import hub
# from langchain.chat_models import ChatOpenAI
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# import pinecone
# from pinecone import Pinecone




In [11]:
# # Initialize a Pinecone client with your API key
# pc = Pinecone(api_key="pcsk_4qt3ZT_U7hLvXByvnhPbktY96YWtg4vh4TQqURmWBWHuLJZquBFQTBNXaaayfbPo8G7ZmT")

# # Create a dense index with integrated embedding
# index_name = "sap-onboarding-index"
# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
#         cloud="aws",
#         region="us-east-1",
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"}
#         }
#     )


In [12]:
# records = [
#     {"_id": "rec1", "chunk_text": "What is SAP used for? SAP is software that helps companies manage business activities like finance, sales, supply chain, and HR. It keeps all important data in one system so different teams can work together smoothly.", "category": "SAP Overview"},
#     {"_id": "rec2", "chunk_text": "What is SAP HANA? SAP HANA is a fast database that stores company information in memory. This makes reports and analytics much faster compared to traditional databases.", "category": "SAP Overview"},
#     {"_id": "rec3", "chunk_text": "What is the difference between SAP ECC and SAP S/4HANA? SAP ECC is the older version of SAP's ERP system. SAP S/4HANA is the newer version, designed to be faster and simpler, and it runs only on the HANA database.", "category": "SAP Overview"},
#     {"_id": "rec4", "chunk_text": "What does 'in-memory' mean in SAP HANA? It means data is stored in the computer's memory (RAM) instead of on a hard disk. This allows SAP to process data in real time, like generating instant reports.", "category": "SAP Overview"},
#     {"_id": "rec5", "chunk_text": "What is SAP Fiori? SAP Fiori is a modern, user-friendly way to use SAP applications. It looks like apps on your phone, making it easier to navigate compared to the older SAP screens.", "category": "SAP Interface"},
#     {"_id": "rec6", "chunk_text": "How does SAP keep data secure? SAP uses login credentials, role-based permissions, and data encryption. This ensures employees only see the information they need for their job.", "category": "SAP Security"},
#     {"_id": "rec7", "chunk_text": "How do I access the company email system? New employees can access the company email via Outlook using your corporate credentials. Set up instructions are in the IT onboarding guide.", "category": "IT & Tools"},
#     {"_id": "rec8", "chunk_text": "Who should I contact for HR-related questions? For questions about benefits, leave, or payroll, contact the HR team at hr@company.com or check the HR portal.", "category": "HR"},
#     {"_id": "rec9", "chunk_text": "How do I log into the internal SAP system? Use your employee ID and temporary password provided during onboarding. After first login, you’ll be prompted to set a new password.", "category": "SAP Access"},
#     {"_id": "rec10", "chunk_text": "Where can I find company policies and guidelines? All company policies are available in the internal employee portal under 'Documents & Policies'.", "category": "Policies"},
#     {"_id": "rec11", "chunk_text": "Who do I contact if I have technical issues? Contact the IT helpdesk at it-support@company.com or submit a ticket via the support portal for assistance.", "category": "IT & Support"}
# ]


In [13]:
# dense_index = pc.Index(index_name)

In [14]:
# dense_index.upsert_records("sap-onboarding", records)

In [15]:
# time.sleep(5)


In [16]:
# # View index stats
# stats = dense_index.describe_index_stats()
# print(stats)

# # Search the index
# query = "How do I access SAP and company tools?"
# results = dense_index.search(
#     namespace="sap-onboarding",
#     query={
#         "top_k": 10,
#         "inputs": {"text": query}
#     }
# )

# # Print results
# for hit in results['result']['hits']:
#     print(f"id: {hit['_id']:<5} | score: {round(hit['_score'],2):<5} | category: {hit['fields']['category']:<15} | text: {hit['fields']['chunk_text'][:80]}...")


In [17]:
# reranked_results = dense_index.search(
#     namespace="sap-onboarding",
#     query={
#         "top_k": 10,
#         "inputs": {"text": query}
#     },
#     rerank={
#         "model": "bge-reranker-v2-m3",
#         "top_n": 10,
#         "rank_fields": ["chunk_text"]
#     }
# )

# for hit in reranked_results['result']['hits']:
#     print(f"id: {hit['_id']}, score: {round(hit['_score'], 2)}, category: {hit['fields']['category']}, text: {hit['fields']['chunk_text'][:80]}...")


In [18]:
# query = "How do I log into SAP?"
# results = dense_index.search(
#     namespace="sap-onboarding",
#     query={"top_k": 5, "inputs": {"text": query}}
# )

# for hit in results['result']['hits']:
#     print(hit['_id'], hit['_score'], hit['fields']['chunk_text'][:80])
